<a href="https://colab.research.google.com/github/Inzamam1234/NLP_ToolKit_FOSS/blob/main/NLP_Toolkit_Grammer_Correction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets

from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import torch

model_name = "t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset ,concatenate_datasets ,DatasetDict

huggingface_dataset_name = "jhu-clsp/jfleg"

dataset = load_dataset(huggingface_dataset_name)

dataset

README.md: 0.00B [00:00, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/141k [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/755 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/748 [00:00<?, ? examples/s]

DatasetDict({
    validation: Dataset({
        features: ['sentence', 'corrections'],
        num_rows: 755
    })
    test: Dataset({
        features: ['sentence', 'corrections'],
        num_rows: 748
    })
})

In [ ]:
combined_dataset = concatenate_datasets([dataset['validation'], dataset['test']])

train_temp_split = combined_dataset.train_test_split(test_size=0.15)

temp_split = train_temp_split['test'].train_test_split(test_size=0.33)

dataset = DatasetDict({
    'train': train_temp_split['train'],
    'validation': temp_split['train'],
    'test': temp_split['test']
})

dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'corrections'],
        num_rows: 1277
    })
    validation: Dataset({
        features: ['sentence', 'corrections'],
        num_rows: 151
    })
    test: Dataset({
        features: ['sentence', 'corrections'],
        num_rows: 75
    })
})

In [ ]:
dataset['train'][0]

{'sentence': 'If the status of cars is still at the present level , something else of course will take the place of cars in our lives , but history tell us that advancement will never stop . ',
 'corrections': ['Even if the status of cars are still at the presently viewed as viable , something else will , of course , take the place of cars in our lives , and history tells us that advancement will never stop . ',
  'If the status of cars is still at the present level , something else will naturally take the place of cars in our lives , but history tells us that advancement will never stop . ',
  'If the status of cars is still at the present level , something else of course will take the place of cars in our lives , but history tell us that advancement will never stop . ',
  'If the status of cars is still at the present level , something else of course will take the place of cars in our lives , but history tells us that advancement will never stop . ']}

In [ ]:
def simlify_correction(example):
    # Flatten the corrections list
    example['corrections'] = [correction for corrections in example["corrections"] for correction in corrections]

    # Repeat each sentence 4 times
    repeated_sentences = []
    for sentence in example['sentence']:
        repeated_sentences.extend([sentence] * 4)
    example['sentence'] = repeated_sentences

    return example

dataset = dataset.map(simlify_correction, batched=True)

print(dataset)

Map:   0%|          | 0/1277 [00:00<?, ? examples/s]

Map:   0%|          | 0/151 [00:00<?, ? examples/s]

Map:   0%|          | 0/75 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'corrections'],
        num_rows: 5108
    })
    validation: Dataset({
        features: ['sentence', 'corrections'],
        num_rows: 604
    })
    test: Dataset({
        features: ['sentence', 'corrections'],
        num_rows: 300
    })
})


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
model_name='t5-base'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16).to('cuda')
full_tuned_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16).to('cuda')
original_peft_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16).to('cuda')

tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=32, # Rank
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=32, # Rank
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

peft_model = get_peft_model(original_peft_model,
                            lora_config)
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 3538944
all model parameters: 226442496
percentage of trainable model parameters: 1.56%


In [ ]:
def preprocess_function(examples):
    # Add the prefix "grammar correction: " to the input sentences
    inputs = ["grammar correction: " + ex for ex in examples["sentence"]]
    # Tokenize the inputs and targets
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
    labels = tokenizer(text_target=examples["corrections"], max_length=128, truncation=True, padding="max_length")

    # Assign the tokenized labels to the input features
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)

print(tokenized_datasets)

Map:   0%|          | 0/5108 [00:00<?, ? examples/s]

Map:   0%|          | 0/604 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'corrections', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 5108
    })
    validation: Dataset({
        features: ['sentence', 'corrections', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 604
    })
    test: Dataset({
        features: ['sentence', 'corrections', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 300
    })
})


In [ ]:
import time
import wandb
from transformers import TrainingArguments, Trainer

output_dir = f'./peft-grammar-correction-training-{str(int(time.time()))}'

wandb.finish()
wandb.init(project="Grammar Correction", name="peft_tuning")

peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-3, # Higher learning rate than full fine-tuning.
    num_train_epochs=1,
    logging_steps=100,
    run_name='peft_tuning_run'
)
peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets['validation']
)

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: injamami85 (injamami85-srm-institute-of-science-and-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
peft_trainer.train()

Step,Training Loss
100,1.110700
200,0.111100
300,0.111800
400,0.099900
500,0.103900
600,0.104200


TrainOutput(global_step=639, training_loss=0.26323822928892804, metrics={'train_runtime': 288.3139, 'train_samples_per_second': 17.717, 'train_steps_per_second': 2.216, 'total_flos': 791522305376256.0, 'train_loss': 0.26323822928892804, 'epoch': 1.0})

In [ ]:
from transformers import GenerationConfig

index = 30
sentence = dataset['test'][index]['sentence']
human_baseline_correction = dataset['test'][index]['corrections']

prompt = f"""
Correct the grammatical mistakes in the following sentence.

{sentence}

Correction: """

input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to('cuda')

original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

dash_line = '-'*100
print(dash_line)
print(f'BASELINE HUMAN CORRECTION:\n{human_baseline_correction}')
print(dash_line)
print(f'ORIGINAL MODEL:\n{original_model_text_output}')
print(dash_line)
print(f'PEFT MODEL:\n{peft_model_text_output}')

----------------------------------------------------------------------------------------------------
BASELINE HUMAN CORRECTION:
It 's so expensive that only those people can do this besides their everyday living . 
----------------------------------------------------------------------------------------------------
ORIGINAL MODEL:
Correction:
----------------------------------------------------------------------------------------------------
PEFT MODEL:
Correct the grammatical mistakes in the following sentence . It is so expensive that only those people can do this besides of their daily living .


In [ ]:
text = "They was playing in the ground."
input_ids = tokenizer(text, return_tensors="pt").input_ids.to('cuda')
peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

print(f'PEFT MODEL:\n{peft_model_text_output}')

PEFT MODEL:
They were playing in the ground.


In [ ]:
text = input("Enter a sentence for grammar correction: ")
input_ids = tokenizer(text, return_tensors="pt").input_ids.to('cuda')
peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

print(f'PEFT MODEL:\n{peft_model_text_output}')

Enter a sentence for grammar correction: i is good
PEFT MODEL:
I am good at reading and writing.


In [ ]:
def save_model_locally(model, tokenizer, save_path="./t5-grammar-model"):
    """Save model and tokenizer to local directory"""
    import os

    # Create directory if it doesn't exist
    os.makedirs(save_path, exist_ok=True)

    # Save model and tokenizer
    model.save_pretrained(save_path)
    tokenizer.save_pretrained(save_path)

    print(f"✅ Model saved to: {save_path}")
    return save_path

save_path = save_model_locally(model, tokenizer)

✅ Model saved to: ./t5-grammar-model


In [ ]:
# Save and download the model locally
save_path = save_model_locally(model, tokenizer)

✅ Model saved to: ./t5-grammar-model


In [ ]:
import zipfile, os
from google.colab import files

#Create ZIP and download in one go
with zipfile.ZipFile('my-model-grammer.zip', 'w') as z:
   for f in os.listdir('/content/t5-grammar-model'): z.write(f'/content/t5-grammar-model/{f}', f)

files.download('my-model-grammer.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>